<h1 align='center'>Welcome to my Notebook</h1>

In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier

from sklearn.model_selection import StratifiedKFold,KFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [15]:
csl_df = pd.read_csv('../../data/csl/CSL_tytl_PI.csv', index_col=0)
X = csl_df.drop(['trans_loss', 'transfus_yes'], axis=1, inplace=False)
y = csl_df['trans_loss'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y,
                                                    random_state=7)

<h1 align='center'>Exploratory Data Analysis</h1>

In [16]:
X

,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,Analgesia,...,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI,Sitenum
MomID,,,,,,,,,,,,,,,,,,,,,
41-00002,0,8,8,8,99.0,83,0.0,77,150.0,7,...,0,0,0,1,0,37,33,1,1,41
41-00003,0,6,3,2,2.9,77,70.0,1,105.0,8,...,0,0,0,0,0,36,22,1,1,41
41-00004,0,5,3,2,3.3,65,70.0,1,122.0,8,...,0,0,0,0,0,34,31,1,1,41
41-00005,0,11,2,3,4.0,77,80.0,77,138.0,8,...,0,0,0,0,0,31,21,1,1,41
41-00006,0,11,1,3,5.0,98,100.0,77,134.0,8,...,0,0,0,0,0,21,22,1,1,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52-07247,0,8,8,8,77.0,74,70.0,1,124.4,5,...,0,0,0,0,0,19,38,0,1,52
52-07248,0,8,8,8,99.0,74,0.0,2,124.4,5,...,0,0,0,1,0,34,24,1,1,52
52-07249,0,8,8,8,66.0,74,0.0,77,124.4,9,...,0,0,0,1,0,28,23,1,1,52


In [17]:
csl_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185413 entries, 41-00002 to 52-07251
Columns: 197 entries, transfus_yes to Sitenum
dtypes: float64(4), int64(193)
memory usage: 280.1+ MB


In [18]:
X.skew()

Accrete           40.650960
AdmBishop         -1.136977
Admcervpos        -0.455299
Admconsistency    -0.407357
Admcontract        0.231978
                    ...    
new_age            0.232310
new_BMI            2.086880
new_high_Age      -0.673167
new_high_BMI      -0.549007
Sitenum           -0.027123
Length: 195, dtype: float64

<h1 align='center'>Categorical Features Data Analysis</h1>

In [19]:
%precision %.3f
#np.set_printoptions(precision=3)
print(f'{np.bincount(y)[1]/len(y):.4f}')
print(f'{np.bincount(y_train)[1]/len(y_train):.4f}')

0.0558
0.0558


In [1]:
fig1, ax1 = plt.subplots()
ax1.pie(csl_df['trans_loss'].value_counts(), labels=['No','Yes'], autopct='%1.1f%%')
plt.show()

NameError: name 'plt' is not defined

In [10]:
#fig = px.pie(train,values=train['Response'].value_counts(),names=['Class 0','Class 1'],hole=0.6,labels={0:'Response = 0'},color_discrete_sequence=px.colors.sequential.Sunset)
#fig.show(showlegend=True)
#plt.show()

In [11]:
#fig = px.pie(train,values=train['Response'].value_counts(),names=['Class 0','Class 1'],hole=0.6,labels={0:'Response = 0'},color_discrete_sequence=px.colors.sequential.Sunset)
#fig.show(showlegend=True)

In [2]:
sns.countplot(csl_df['trans_loss'])
plt.show()

NameError: name 'sns' is not defined

We can see from the above visualisations that:

* The Data is highly imbalanced

In [3]:
plt.figure(figsize=(20,10))
sns.barplot(csl_df['new_age'],csl_df['trans_loss'])
plt.show()

NameError: name 'plt' is not defined

In [4]:
sns.boxplot(csl_df['new_age'])
plt.show()

NameError: name 'sns' is not defined

In [5]:
plt.figure(figsize=(20,10))
sns.barplot(csl_df['new_BMI'],csl_df['trans_loss'])
plt.show()

NameError: name 'plt' is not defined

In [6]:
sns.boxplot(csl_df['new_BMI'])
plt.show()

NameError: name 'sns' is not defined

<h1 align='center'>Modelling</h1>

In [19]:
#X = all_features.iloc[:len(train),:]
#X_test = all_features.iloc[len(train):,:]

#kf = StratifiedKFold(n_splits=12,shuffle=True,random_state=42)

We define resplit our train and test set, and set up 12 Fold Stratified Cross Validation. It is important to stratify as this ensures that during our training and validation, we split according to target distribution

In [20]:
#for train_index,val_index in kf.split(X,y):
#    X_train,X_val = X.iloc[train_index],X.iloc[val_index],
#    y_train,y_val = y.iloc[train_index],y.iloc[val_index],

In [25]:
# MWB
print(X_train.shape)
print(X.shape)
print(y.shape)

(129789, 195)
(185413, 195)
(185413,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [26]:
#sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
#X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
X_rus, y_rus = X_train, y_train

In [27]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(129789, 195)
(129789,)
[122548   7241]


<h1 align='center'>Basic LightGBM</h1>

Let's fit a vanilla LGBMClassifier on the undersampled data and evaluate it

In [29]:
from sklearn.metrics import matthews_corrcoef
lgb_rus = LGBMClassifier(class_weight='balanced')   # If not undersampling
# lgb_rus = LGBMClassifier()
lgb_rus.fit(X_rus, y_rus)
y_pred = lgb_rus.predict(X_test)
probs = lgb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[35561 16960]
 [  373  2730]]
              precision    recall  f1-score   support

           0       0.99      0.68      0.80     52521
           1       0.14      0.88      0.24      3103

    accuracy                           0.69     55624
   macro avg       0.56      0.78      0.52     55624
weighted avg       0.94      0.69      0.77     55624

ROC AUC prob1: 0.8404317845625434
MCC: 0.26726352596949493
F2: 0.42520715220235505
PR_AUC: 0.23579169084608326


<h1 align='center'>Hyperparameter Tuning</h1>

In [32]:
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators',100,500)
    num_leaves = trial.suggest_int('num_leaves',10,500)
    max_depth = trial.suggest_int('max_depth',4,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.0001,1)
    min_child_samples = trial.suggest_int('min_child_samples',10,50)
    model = LGBMClassifier(n_estimators=n_estimators, num_leaves=num_leaves, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_child_samples=min_child_samples, class_weight='balanced')
#    model = LGBMClassifier(n_estimators=n_estimators, num_leaves=num_leaves, 
#                           max_depth=max_depth, learning_rate=learning_rate, 
#                           min_child_samples=min_child_samples)
    return model

def objective(trial):
    model = create_model(trial)
    model.fit(X_rus, y_rus)
    y_pred = model.predict(X_test)
    probs = model.predict_proba(X_test)
    prob1 = probs[:, 1]  # Only positives
    precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
    roc = roc_auc_score(y_test, prob1)
    mcc = matthews_corrcoef(y_test, y_pred)  # MWB
    f2 = fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]
    pr_auc = auc(recall, precision)
    avg_precision = average_precision_score(y_test, prob1)
    print(f'PR_AUC: {pr_auc}')
    print(f'avg_prec: {avg_precision}')
    print(f'MCC: {mcc}')
    return mcc
#    return roc

sampler = TPESampler(seed=42)
study = optuna.create_study(sampler=sampler,direction='maximize')
study.optimize(objective,n_trials=60)
#study.optimize(objective,n_trials=10)

[I 2021-12-02 18:08:40,330] A new study created in memory with name: no-name-0efee87c-7a33-4b9e-b73c-d463f5fcfe60
[I 2021-12-02 18:08:50,336] Trial 0 finished with value: 0.10985953775151971 and parameters: {'n_estimators': 250, 'num_leaves': 476, 'max_depth': 16, 'learning_rate': 0.5986986183486169, 'min_child_samples': 16}. Best is trial 0 with value: 0.10985953775151971.


PR_AUC: 0.16571953411951862
avg_prec: 0.16590258540184355
MCC: 0.10985953775151971


[I 2021-12-02 18:08:52,690] Trial 1 finished with value: 0.17760394043883007 and parameters: {'n_estimators': 162, 'num_leaves': 38, 'max_depth': 18, 'learning_rate': 0.6011549002420344, 'min_child_samples': 39}. Best is trial 1 with value: 0.17760394043883007.


PR_AUC: 0.16461964020142256
avg_prec: 0.16494590720538013
MCC: 0.17760394043883007


[I 2021-12-02 18:08:57,257] Trial 2 finished with value: 0.13252223067132346 and parameters: {'n_estimators': 108, 'num_leaves': 486, 'max_depth': 18, 'learning_rate': 0.21241787676720833, 'min_child_samples': 17}. Best is trial 1 with value: 0.17760394043883007.


...

PR_AUC: 0.15002475412226424
avg_prec: 0.15033606454238585
MCC: 0.1585649887977986


[I 2021-12-02 18:12:17,286] Trial 58 finished with value: 0.26178002727794497 and parameters: {'n_estimators': 140, 'num_leaves': 455, 'max_depth': 5, 'learning_rate': 0.18868136930475382, 'min_child_samples': 19}. Best is trial 26 with value: 0.2681359505444071.


PR_AUC: 0.22579327951961664
avg_prec: 0.22600257947856595
MCC: 0.26178002727794497


[I 2021-12-02 18:12:19,869] Trial 59 finished with value: 0.2644800625200721 and parameters: {'n_estimators': 116, 'num_leaves': 62, 'max_depth': 9, 'learning_rate': 0.0602913571310376, 'min_child_samples': 13}. Best is trial 26 with value: 0.2681359505444071.


PR_AUC: 0.23493339150990936
avg_prec: 0.23510834931726055
MCC: 0.2644800625200721


In [31]:
lgb_params = study.best_params
lgb_params['random_state'] = 42
lgb_params['class_weight'] = 'balanced'
lgb = LGBMClassifier(**lgb_params)
lgb.fit(X_rus, y_rus)
y_pred = lgb.predict(X_test)
probs = lgb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test,lgb.predict(X_test)))
print(f'ROC AUC pred: {roc_auc_score(y_test, y_pred)}')
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[35311 17210]
 [  355  2748]]
              precision    recall  f1-score   support

           0       0.99      0.67      0.80     52521
           1       0.14      0.89      0.24      3103

    accuracy                           0.68     55624
   macro avg       0.56      0.78      0.52     55624
weighted avg       0.94      0.68      0.77     55624

ROC AUC pred: 0.7789580667280378
ROC AUC prob1: 0.8413393017944365
MCC: 0.26695655516312067
F2: 0.4244670991658943
PR_AUC: 0.23720579120809138


In [35]:
lgb_params = study.best_params
lgb_params['random_state'] = 42
lgb_params['class_weight'] = 'balanced'
lgb = LGBMClassifier(**lgb_params)
lgb.fit(X_rus, y_rus)
y_pred = lgb.predict(X_test)
probs = lgb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test,lgb.predict(X_test)))
print(f'ROC AUC pred: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[35321 17200]
 [  348  2755]]
              precision    recall  f1-score   support

           0       0.99      0.67      0.80     52521
           1       0.14      0.89      0.24      3103

    accuracy                           0.68     55624
   macro avg       0.56      0.78      0.52     55624
weighted avg       0.94      0.68      0.77     55624

ROC AUC pred: 0.8413901232012144
MCC: 0.2681359505444071
F2: 0.42558779003305836
PR_AUC: 0.23328128665994488


In [37]:
lgb_params

{'n_estimators': 214,
 'num_leaves': 378,
 'max_depth': 4,
 'learning_rate': 0.079,
 'min_child_samples': 37,
 'random_state': 42,
 'class_weight': 'balanced'}

<h1 align='center'>Thanks and make sure to learn!</h1>